# Neste notebook faremos a simulação por dinâmica molecular do gás argônio superfluido à 100 K e 1000 atm.

## Etapa 1
O primeiro passo consiste na instalação do pacote que será responsável pela dinâmica molecular. Descomente a linha abaixo caso o pacote M3L ainda não foi instalado no seu ambiente virtual python numa versão superior à 3.11.

In [1]:
#pip install -i https://test.pypi.org/simple/ m3l==2025.4.dev6 --extra-index-url https://pypi.org/simple/

## Etapa 2
O segundo passo consiste no carregamento do sistema físico contendo as coordenadas atômicas e suas propriedades termodinâmicas.

In [2]:
# download do arquivo 'Ar_100K_1000atm.json'
url = 'https://raw.githubusercontent.com/flavianowilliams/M3L/development/src/notebooks/Ar_100K_1000atm.json'
from urllib.request import urlopen
#
#file = urlopen(url)
#
#with open('Ar_100K_1000atm.json','wb') as output:
#  output.write(file.read())

In [3]:
# carregando sistema físico
from m3l.structure import System as sys
import json
#
system = sys()
system.loadSystem('Ar_100K_1000atm.json')
#
# convertendo as unidades de medida
system.convertUnits()

## Etapa 3
O terceiro passo consiste em definir o modelo para o campo de força.

In [4]:
# definindo o modelo de interação entre os átomos (campo de força)
from m3l.molecular_dynamics import ForceField
class Forces(ForceField):
    def __init__(self):
        super().__init__()
        self.van_der_waals(
            (1, 0.2381*self.ECONV, 3.405*self.ACONV)
        )
#        
model = Forces()

## Etapa 4
O quarto passo consiste em definir o modelo estatístico (ensemble canônico) que descreverá o sistema no equilíbrio termodinâmico.

In [5]:
# definindo o modelo estatístico (ensemble)
from m3l.molecular_dynamics import Ensemble as en
#
force_field = model()
ensemble = en(temp_bath = 100.0, press_bath = 1000.0, timestep = 1.0e-4, force_field = force_field, tstat = 1.0, pstat = 1.0, bfactor = 4.9e-5)

## Etapa 5
O quinto passo consiste na dinâmica molecular realizada em looping dentro de uma quantidade definida de passos.

In [6]:
import time
#
# executando looping
n_steps = 1000
i_step = 0
start = time.time()
for step in range(n_steps):
    system = ensemble(system)
end = time.time()
print(f'Elapsed time: {round(end - start, 0)} seconds')

Elapsed time: 2.0 seconds


## Etapa 6
O sexto e último passo o autor poderá salvar em um arquivo JSON o sistema físico obtido no final da simulação.

In [7]:
# convertendo para as unidades de medida de entrada
system.convertUnitsInv()
#
# salvando sistema em arquivo JSON
system.save('system_new.json')

In [8]:
system.temperature

np.float64(96.94657711021149)